In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from flask import Flask, request, jsonify, render_template

In [2]:
animedf = pd.read_csv("Dataset/anime.csv")

ratingsdf = pd.read_csv("Dataset/rating.csv")

In [3]:
anime = animedf.dropna()

In [4]:
count = animedf['rating'].value_counts()

animedf = animedf[animedf['rating'].isin(count[count>=4].index)]



In [5]:
#our datasets after conditional selection 
animedf.rename(columns = {'rating':'animeRating'}, inplace=True)


In [6]:
ratingsdf.rename(columns={'rating':'userRating'}, inplace=True)

In [7]:
combineAnime = pd.merge(ratingsdf,animedf, on ='anime_id')

In [8]:
combineAnime = combineAnime.dropna()

In [9]:
animeRatingCount = (combineAnime.
                     groupby(by=['name'])['animeRating'].
                     count().
                     reset_index().
                     rename(columns = {'animeRating': 'totalRatingCount'})
                     [['name','totalRatingCount']]
                    )


In [10]:
animeRatingCount['name'] = animeRatingCount['name'].apply(lambda x : x.replace('&quot;',''))

In [11]:
finalCombineAnime = combineAnime.merge(animeRatingCount,left_on='name', right_on='name',how='left')


In [12]:
popularity_threshold = 500

rating_popular_anime = finalCombineAnime.query('totalRatingCount>= @popularity_threshold')

rating_popular_anime['name'] = rating_popular_anime['name'].str.lower()

C:\Users\91847\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
from scipy.sparse import csr_matrix

#We will be dropping duplicates in order to make it efficient from both bookTitle and userID

rating_popular_anime = rating_popular_anime.drop_duplicates(['user_id','name'])

#This setp is important as we will be creating pivot table for the final dataframe 

rating_popular_anime_pivot = rating_popular_anime.pivot(index='name', columns='user_id', values ='animeRating').fillna(0)

rating_popular_anime_matrix = csr_matrix(rating_popular_anime_pivot.values)


#We will be training our data by fitting it into nearest neighbour

from sklearn.neighbors import NearestNeighbors

model_knn  = NearestNeighbors(metric='cosine', algorithm='brute')

model_knn.fit(rating_popular_anime_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [14]:
app = Flask(__name__)

In [15]:
@app.route('/')
def home():
    return render_template('index.html')

In [16]:

def rcm(m):
    
    
    m = m.lower()
    
    if name in rating_popular_anime_pivot.index:
        userquery = m
        distances,indices  = model_knn.kneighbors(rating_popular_anime_pivot.loc[userquery,:].values.reshape(1,-1),
                                         n_neighbors=6)
        animelist = []
        
        for i in range(0, len(distances.flatten())):
            if i == 0:
                print('Recommendations for {0}:\n'.format(userquery))
            else:
               
        
                x = rating_popular_anime_pivot.index[indices.flatten()[i]]
                animelist.append(x)
        return animelist
    


    else:
        return ('Sorry, either it is a typo, or the searched entity is not avaliable, which is rare')
    

    
    
    

In [17]:
@app.route('/home',methods=['POST'])

def prediction():
    anime = request.args.get('anime')
    r = rcm(anime)
    if type(r)==type('string'):
        return render_template('predictions.html',movie=movie,r=r,t='s')
    else:
        return render_template('predictions.html',movie=movie,r=r,t='l')

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-08-19 19:07:10,851] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\91847\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-15-bf5f